<a href="https://colab.research.google.com/github/elyorbek8/ML_journey/blob/main/step6_pipleine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creating pipelines

Pipeline is a structured workflow that automates the process.


In [1]:
import pandas as pd
import numpy as np
import sklearn

In [2]:
df = pd.read_csv('https://github.com/ageron/handson-ml2/blob/master/datasets/housing/housing.csv?raw=true')
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


# Spliting train and test sets

In [4]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(df, test_size= 0.2, random_state= 42)

In [5]:
# data, labels, and numerical data
housing = train_set.drop('median_house_value', axis= 1)
housing_labels = train_set['median_house_value'].copy()

housing_num = housing.drop('ocean_proximity', axis= 1)

# our transformer: adding extra columns - ```rooms_per_household```, ```bedrooms_per_household```, and ```population_per_household```.

In [6]:
from sklearn.base import BaseEstimator, TransformerMixin

# indices
rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6

class AttributeAdder(BaseEstimator, TransformerMixin):
  def __init__(self, add_bedrooms_per_household = True):
    self.add_bedrooms_per_household = add_bedrooms_per_household

  def fit(self, X, y= None):
    return self

  def transform(self, X):
    rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
    population_per_household = X[:, population_ix] / X[:, households_ix]
    if self.add_bedrooms_per_household:
      bedrooms_per_household = X[:, bedrooms_ix] / X[:, households_ix]
      return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_household]
    else:
      return np.c_[X, rooms_per_household, population_per_household]

# Creating pipelines for replacing NaN values, encoding, standardization

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [8]:
# numeric pipeline
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy= 'median')),
    ('attr_adder', AttributeAdder()),
    ('std_scaler', StandardScaler())
])

num_pipeline.fit_transform(housing_num)

array([[ 1.27258656, -1.3728112 ,  0.34849025, ..., -0.17491646,
         0.05137609, -0.20836543],
       [ 0.70916212, -0.87669601,  1.61811813, ..., -0.40283542,
        -0.11736222, -0.12853018],
       [-0.44760309, -0.46014647, -1.95271028, ...,  0.08821601,
        -0.03227969, -0.25753771],
       ...,
       [ 0.59946887, -0.75500738,  0.58654547, ..., -0.60675918,
         0.02030568, -0.03921583],
       [-1.18553953,  0.90651045, -1.07984112, ...,  0.40217517,
         0.00707608, -0.06626528],
       [-1.41489815,  0.99543676,  1.85617335, ..., -0.85144571,
        -0.08535429, -0.08750798]])

# Creating pipeline that can include another pipeline itself.

In [18]:
from sklearn.compose import ColumnTransformer

num_attr = list(housing_num)
cat_attr = ['ocean_proximity']

full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attr),
    ('cat', OneHotEncoder(), cat_attr)
])

In [20]:
housing_done = full_pipeline.fit_transform(housing)

In [26]:
housing_done[:5, :]

array([[ 1.27258656, -1.3728112 ,  0.34849025,  0.22256942,  0.21122752,
         0.76827628,  0.32290591, -0.326196  , -0.17491646,  0.05137609,
        -0.20836543,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ],
       [ 0.70916212, -0.87669601,  1.61811813,  0.34029326,  0.59309419,
        -0.09890135,  0.6720272 , -0.03584338, -0.40283542, -0.11736222,
        -0.12853018,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ],
       [-0.44760309, -0.46014647, -1.95271028, -0.34259695, -0.49522582,
        -0.44981806, -0.43046109,  0.14470145,  0.08821601, -0.03227969,
        -0.25753771,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ],
       [ 1.23269811, -1.38217186,  0.58654547, -0.56148971, -0.40930582,
        -0.00743434, -0.38058662, -1.01786438, -0.60001532,  0.07750687,
        -0.14515634,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ],
       [-0.10855122,  0.5320839 ,  1